In [5]:
import pdfplumber
import re
import os
import json

def smart_clean_text(text: str) -> str:
    """
    문맥을 고려하여 줄바꿈을 지능적으로 처리하는 함수입니다.
    - 문단 끝 줄바꿈은 공백으로 변환합니다.
    - 단어 중간의 줄바꿈은 제거하여 단어를 합칩니다.
    """
    # 1. 문단 구분을 위한 두 번 이상의 줄바꿈을 임시 문자로 치환
    text = re.sub(r'(\n\s*){2,}', ' __PARA_BREAK__ ', text)
    
    # 2. 문장 중간의 단순 줄바꿈은 제거
    text = text.replace('\n', '')
    
    # 3. 임시 문자를 다시 공백으로 변환하여 문단 구분
    text = text.replace('__PARA_BREAK__', ' ')
    
    # 4. 여러 개의 연속된 공백을 하나로 합침
    clean_text = re.sub(r'\s+', ' ', text).strip()
    
    return clean_text

def process_pdfs_with_smart_cleaning(pdf_paths: list, max_chunk_size: int) -> list:
    """
    pdfplumber로 텍스트를 추출하고, 스마트 정제 로직을 적용하여
    최종 텍스트 청크를 생성합니다.
    """
    all_chunks = []
    chunk_id_counter = 1

    for pdf_path in pdf_paths:
        if not os.path.exists(pdf_path):
            print(f"경고: '{pdf_path}' 파일을 찾을 수 없습니다.")
            continue

        print(f"'{pdf_path}' 파일을 스마트 정제 방식으로 처리 중...")
        full_doc_text = ""

        with pdfplumber.open(pdf_path) as pdf:
            for page in pdf.pages:
                page_text = page.extract_text()
                if page_text:
                    full_doc_text += page_text + "\n\n" # 페이지 끝에 문단 구분을 추가

        # --- 새로 추가된 '스마트 정제' 함수 호출 ---
        clean_text = smart_clean_text(full_doc_text)

        # 문장 단위로 분할 및 청크 생성 로직은 동일
        sentences = re.split(r'(?<=[.?!])\s+', clean_text)
        current_chunk = ""
        for sentence in sentences:
            if not sentence.strip(): continue

            if len(current_chunk) + len(sentence) + 1 > max_chunk_size:
                if current_chunk:
                    all_chunks.append({
                        "id": f"{os.path.basename(pdf_path).split('.')[0]}-{chunk_id_counter}",
                        "source_document": os.path.basename(pdf_path),
                        "content": current_chunk.strip()
                    })
                    chunk_id_counter += 1
                current_chunk = sentence
            else:
                current_chunk += " " + sentence if current_chunk else sentence
        
        if current_chunk:
            all_chunks.append({
                "id": f"{os.path.basename(pdf_path).split('.')[0]}-{chunk_id_counter}",
                "source_document": os.path.basename(pdf_path),
                "content": current_chunk.strip()
            })
            chunk_id_counter += 1

    return all_chunks

if __name__ == "__main__":
    pdf_files_to_process = ["hyundai_design_philosophy1.pdf"]
    MAX_CHUNK_CHAR_SIZE = 500

    structured_db = process_pdfs_with_smart_cleaning(pdf_files_to_process, max_chunk_size=MAX_CHUNK_CHAR_SIZE)

    output_filename = "hyundai_design_db_smart_cleaned.json"
    with open(output_filename, 'w', encoding='utf-8') as f:
        json.dump(structured_db, f, ensure_ascii=False, indent=4)

    print(f"\n✅ 스마트 정제 완료! 총 {len(structured_db)}개의 청크가 생성되었습니다.")
    print(f"✅ 결과가 '{output_filename}' 파일에 저장되었습니다.")

    print("\n--- 최종 결과물 샘플 (첫 3개) ---")
    for item in structured_db[:3]:
        print(json.dumps(item, ensure_ascii=False, indent=2))
        print("-" * 20)

'hyundai_design_philosophy1.pdf' 파일을 스마트 정제 방식으로 처리 중...

✅ 스마트 정제 완료! 총 45개의 청크가 생성되었습니다.
✅ 결과가 'hyundai_design_db_smart_cleaned.json' 파일에 저장되었습니다.

--- 최종 결과물 샘플 (첫 3개) ---
{
  "id": "hyundai_design_philosophy1-1",
  "source_document": "hyundai_design_philosophy1.pdf",
  "content": "현대자동차의 디자인 정체성 ‘플루이딕 스컬프처’의전략적 고찰A Strategic Review on ‘Fluidic Sculpture', the Design Identity of HMC이주명연세대학교 인문예술대학 디자인예술학부 교수Rhi Joo-myungYonsei university 를 중심으로 검토하였다.1. 서 론그 결과 현대자동차의 신규 디자인정체성은 ‘세련1.1. 연구 배경되고 당당한’ 현대자동차의 브랜드전략을 반영하고1.2. 연구 목적 및 방법있으며, ‘다이나믹’을 추구해야 하는 자동차의 속성과2. 현대자동차의 브랜드 전략 검토 시장 상황에 전략적으로 적합한 이미지 포지셔닝을취하고 있었다. 또한 핵심 조형 요소들이 종합되어2.1. 디자인 정체성과 브랜드 정체성조형 유산도 대체로 반영되고 있다는 것을 확인하였2.2. 기업 정체성과 브랜드 슬로건2.3. 현대․기아브랜드의 차별화 다. 특히 다소 과도한 것으로 평가받고 있는 현대자동차의 신규 이미지 포지셔닝은 프리미엄 이미지로3."
}
--------------------
{
  "id": "hyundai_design_philosophy1-2",
  "source_document": "hyundai_design_philosophy1.pdf",
  "content": "현대자동차의 조형 유산나아가기 위한 현대의 중간단계 전략으로서 불가피한3.1. 조형 유산의 추출 선택으로 보이지만 향후 점차 정련되어 새로운